In [26]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
from sklearn.model_selection import KFold

import numpy as np
import os
import pandas as pd

import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
import torch.nn as nn

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

In [53]:
# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 3 * 100
num_classes = 3
learning_rate = 0.017287600194096035
batch_size = 10
num_epochs = 5

In [54]:
#Create fully connected Network
class MLP(nn.Module):
    def __init__(self, input_size = input_size, num_classes = num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 25)
        self.fc3 = nn.Linear(25, 10)
        self.fc4 = nn.Linear(10, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [55]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [56]:
#load in the data
###from google.colab import drive
###drive.mount('/content/drive')

In [57]:
#load labels
###labels = np.genfromtxt('/content/drive/MyDrive/ML Project 2/Features/Labels/avg_smooth_5.csv', delimiter=',')
labels = np.genfromtxt('Data/Labels/avg_smooth.csv', delimiter=',')
#print(labels)

In [58]:
#make labels categorical: 6 quantification levels
df_labels = pd.DataFrame(labels)
df_labels = pd.cut(df_labels[0],bins=[0, 0.01, 0.02, 0.03],labels=[0,1,2])
labels = df_labels.to_numpy()
#print(labels)

In [59]:
#load .npy files into one big array
data = []
###dir = '/content/drive/MyDrive/ML Project 2/Features/Feature arrays/'
dir = 'Data/input_MLP/'
pac = np.load('Data/PAC_afterCNN.npy')
for filename in os.listdir(dir):
    sample = np.load(dir+filename)
    data.append(sample.flatten())
data = np.array(data)
#data = np.expand_dims(data,1)
print(data.shape, labels.shape)

(600, 300) (600,)


In [60]:
#transform data to torch tensors
tensor_x = torch.Tensor(data)
tensor_y = torch.Tensor(labels)
tensor_y = tensor_y.type(torch.LongTensor)
###print(tensor_y)

In [61]:
#create dataset and data loader
dataset = TensorDataset(tensor_x,tensor_y)

#split into train and test sets
#treat as hyperparameters later on
#batch_size = 10
test_size = int(0.3*len(dataset))
train_size = len(dataset) - test_size

train_data,test_data = random_split(dataset,[train_size,test_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Test Data : {len(test_data)}")

train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 4, pin_memory = True)
test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 4, pin_memory = True)

Length of Train Data : 420
Length of Test Data : 180


In [62]:
# Initialize network
model = MLP(input_size, num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [63]:
#train network
for epoch in range(num_epochs):
    for batch_idx, (data, labels) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = labels.to(device=device)
        # Get to correct shape
        ###data = data.reshape(data.shape[0], -1)
        ###data = data.reshape(-1, )
        ###data = data.reshape(-1, 3, 110, 40)
        #print(data.shape)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 24.16it/s]


In [64]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 73.10
Accuracy on test set: 65.56


In [68]:
# Define the K-fold Cross Validator
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=False)

In [69]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            #print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [70]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples
train_acc = []
test_acc = []
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold}')
    model.apply(reset_weights)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    trainloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(
                      dataset, batch_size=10, sampler=test_subsampler)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for batch_idx, (input_, labels) in enumerate(tqdm(trainloader)):
            # Get data to cuda if possible
            input_ = input_.to(device=device)
            targets = labels.to(device=device)

            # forward
            scores = model(input_)
            loss = criterion(scores, targets)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()
        train_acc.append(check_accuracy(trainloader, model).item()*100)
        test_acc.append(check_accuracy(testloader, model).item()*100)
    print('Train Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(trainloader, model)))
    print('Test Accuracy for fold %d: %d %%' % (fold, 100.0 * check_accuracy(testloader, model)))
print('Averaged Train Accuracy over %d k-folds: %d %%' % (k_folds, np.array(train_acc).mean()))
print('Averaged Test Accuracy over %d k-folds: %d %%' % (k_folds, np.array(test_acc).mean()))

  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

Fold 0


  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

Train Accuracy for fold 0: 73 %
Test Accuracy for fold 0: 61 %
Fold 1


  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

...

Train Accuracy for fold 2: 69 %
Test Accuracy for fold 2: 76 %
Fold 3



  0%|                                                                                           | 0/48 [00:00<?, ?it/s]

Train Accuracy for fold 3: 73 %
Test Accuracy for fold 3: 58 %
Fold 4


100%|█████████████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 318.76it/s]

Train Accuracy for fold 4: 70 %
Test Accuracy for fold 4: 71 %
[73.12500476837158, 73.12500476837158, 73.12500476837158, 73.12500476837158, 73.12500476837158, 67.08333492279053, 67.08333492279053, 67.08333492279053, 67.08333492279053, 67.08333492279053, 69.37500238418579, 69.37500238418579, 69.37500238418579, 69.37500238418579, 69.37500238418579, 73.95833730697632, 73.95833730697632, 73.95833730697632, 73.95833730697632, 73.95833730697632, 70.6250011920929, 70.6250011920929, 70.6250011920929, 70.6250011920929, 70.6250011920929]
Averaged Train Accuracy over 5 k-folds: 70 %
Averaged Test Accuracy over 5 k-folds: 70 %


In [38]:
config = {
    "num_epochs": tune.choice([2, 5, 10]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([5, 10, 50, 100])
}
max_num_epochs = 11

In [39]:
def train(config, checkpoint_dir=None):
    model = MLP().to(device)
    model.apply(reset_weights)
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    batch_size = config["batch_size"]
    train_loader = DataLoader(train_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    test_loader = DataLoader(test_data, batch_size, shuffle = False, num_workers = 2, pin_memory = True)
    for epoch in range(config["num_epochs"]):
        running_loss = 0
        epoch_steps = 0
        for batch_idx, (data, labels) in enumerate(tqdm(train_loader)):
            current_loss = 0
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = labels.to(device=device)
            # Get to correct shape
            ###data = data.reshape(data.shape[0], -1)
            ###data = data.reshape(-1, )
            ###data = data.reshape(-1, 3, 110, 40)
            #print(data.shape)

            # forward
            scores = model(data)
            loss = criterion(scores, targets)

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent or adam step
            optimizer.step()

            #print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if batch_idx%500 == 499:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0
                
                
        test_loss = 0
        test_steps = 0
        total = 0
        correct = 0
        for batch_idx, (data, labels) in enumerate(tqdm(test_loader)):
            with torch.no_grad():
                data = data.to(device=device)
                targets = labels.to(device=device)
                scores = model(data)
                _, predictions = scores.max(1)
                total += predictions.size(0)
                correct += (predictions == targets).sum().item()
                loss = criterion(scores, targets)
                test_loss += loss.cpu().numpy()
                test_steps+=1
        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)
        tune.report(loss=(test_loss/test_steps), accuracy=correct/total)
    print("Finished")

In [40]:
max_num_epochs = 11
def main(num_samples = 10, gpus_per_trial=1):
    config = {
    "num_epochs": tune.choice([4, 5, 10]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([10, 30, 50, 100])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t = max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(metric_columns=["loss", "accuray", "train_iteration"])
    result = tune.run(
            partial(train),
            resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
            config=config,
            num_samples = num_samples,
            scheduler=scheduler,
            progress_reporter=reporter)
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    #best_trained_model = model(best_trial)

In [41]:
main(num_samples=10, gpus_per_trial=1)

2021-12-18 16:46:07,711	WARNING experiment.py:255 -- No name detected on trainable. Using DEFAULT.
2021-12-18 16:46:07,712	INFO registry.py:69 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Current time: 2021-12-18 16:46:08 (running for 00:00:00.31)
Memory usage on this node: 17.6/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/24 CPUs, 0/2 GPUs, 0.0/38.81 GiB heap, 0.0/19.41 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-46-07
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+--------------+-----------------+--------------+
| Trial name          | status   | loc   |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-------+--------------+-----------------+--------------|
| DEFAULT_9dac2_00000 | PENDING  |       |           50 |     0.00450476  |           10 |
| DEFAULT_9dac2_00001 | PENDING  |       |           50 |     0.0106262   |           10 |
| DEFAULT_9dac2_00002 | PENDING  |       |           50 |     0.0841042   |           10 |
| DEFAULT_9dac2_00003 | PENDIN

  0%|          | 0/9 [00:00<?, ?it/s]


== Status ==
Current time: 2021-12-18 16:46:13 (running for 00:00:05.32)
Memory usage on this node: 18.9/47.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1.0/24 CPUs, 1.0/2 GPUs, 0.0/38.81 GiB heap, 0.0/19.41 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-46-07
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-----------------+--------------+-----------------+--------------+
| Trial name          | status   | loc             |   batch_size |   learning_rate |   num_epochs |
|---------------------+----------+-----------------+--------------+-----------------+--------------|
| DEFAULT_9dac2_00000 | RUNNING  | 127.0.0.1:25644 |           50 |     0.00450476  |           10 |
| DEFAULT_9dac2_00001 | PENDING  |                 |           50 |     0.0106262   |           10 |
| DEFAULT_9dac2_00002 | PENDING  |                 |     

  0%|          | 0/4 [00:00<?, ?it/s]


...

== Status ==
Current time: 2021-12-18 16:50:57 (running for 00:04:49.56)
Memory usage on this node: 17.7/47.6 GiB
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 8.000: -0.8682056963443756 | Iter 4.000: -0.8590151833163368 | Iter 2.000: -0.8739355951547623 | Iter 1.000: -0.9058503177430894
Resources requested: 1.0/24 CPUs, 1.0/2 GPUs, 0.0/38.81 GiB heap, 0.0/19.41 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-46-07
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+-----------------+--------------+-----------------+--------------+-----------+
| Trial name          | status     | loc             |   batch_size |   learning_rate |   num_epochs |      loss |
|---------------------+------------+-----------------+--------------+-----------------+--------------+-----------|
| DEFAULT_9dac2_00009 | RUNNING    | 127.0.0.1:16736 |           50 |     0.000934897 |            4 |           |
| DEFAULT_9dac2_00000 | TERMINATED | 12

  0%|          | 0/9 [00:00<?, ?it/s]


== Status ==
Current time: 2021-12-18 16:51:03 (running for 00:04:55.40)
Memory usage on this node: 19.0/47.6 GiB
Using AsyncHyperBand: num_stopped=5
Bracket: Iter 8.000: -0.8682056963443756 | Iter 4.000: -0.8590151833163368 | Iter 2.000: -0.8739355951547623 | Iter 1.000: -0.9058503177430894
Resources requested: 1.0/24 CPUs, 1.0/2 GPUs, 0.0/38.81 GiB heap, 0.0/19.41 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-46-07
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+-----------------+--------------+-----------------+--------------+-----------+
| Trial name          | status     | loc             |   batch_size |   learning_rate |   num_epochs |      loss |
|---------------------+------------+-----------------+--------------+-----------------+--------------+-----------|
| DEFAULT_9dac2_00009 | RUNNING    | 127.0.0.1:16736 |           50 |     0.000934897 |            4 |           |
| DEFAULT_9dac2_00000 | TERMINATED | 12

100%|██████████| 4/4 [00:03<00:00,  1.12it/s]
2021-12-18 16:51:07,823	INFO tune.py:626 -- Total run time: 300.11 seconds (299.96 seconds for the tuning loop).


Result for DEFAULT_9dac2_00009:
  accuracy: 0.6611111111111111
  date: 2021-12-18_16-51-07
  done: true
  experiment_id: 45df0267756f4a3cbef2fcb404858296
  hostname: WSTOMAS21
  iterations_since_restore: 1
  loss: 0.9628212749958038
  node_ip: 127.0.0.1
  pid: 16736
  should_checkpoint: true
  time_since_restore: 9.583799839019775
  time_this_iter_s: 9.583799839019775
  time_total_s: 9.583799839019775
  timestamp: 1639842667
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9dac2_00009
  
== Status ==
Current time: 2021-12-18 16:51:07 (running for 00:04:59.97)
Memory usage on this node: 19.1/47.6 GiB
Using AsyncHyperBand: num_stopped=6
Bracket: Iter 8.000: -0.8682056963443756 | Iter 4.000: -0.8590151833163368 | Iter 2.000: -0.8739355951547623 | Iter 1.000: -0.9316887838972939
Resources requested: 0/24 CPUs, 0/2 GPUs, 0.0/38.81 GiB heap, 0.0/19.41 GiB objects
Result logdir: C:\Users\Tomas\ray_results\DEFAULT_2021-12-18_16-46-07
Number of trials: 10/10 (10 TERMINATED)
+---